In [1]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [2]:
!./elasticsearch-7.0.0/bin/elasticsearch-plugin install https://github.com/medcl/elasticsearch-analysis-ik/releases/download/v7.0.0/elasticsearch-analysis-ik-7.0.0.zip
!./elasticsearch-7.0.0/bin/elasticsearch-plugin list

-> Downloading https://github.com/medcl/elasticsearch-analysis-ik/releases/download/v7.0.0/elasticsearch-analysis-ik-7.0.0.zip
[=================================================] 100%   
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@     WARNING: plugin requires additional permissions     @
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
* java.net.SocketPermission * connect,resolve
See http://docs.oracle.com/javase/8/docs/technotes/guides/security/permissions.html
for descriptions of what these permissions allow and the associated risks.

Continue with installation? [y/N]y
-> Installed analysis-ik
analysis-ik


In [3]:
!pip install elasticsearch -q

     |████████████████████████████████| 225kB 3.5MB/s 


In [4]:
from subprocess import Popen, PIPE, STDOUT
from elasticsearch import Elasticsearch
from elasticsearch import helpers

import os
import time
import pprint

In [5]:
es_server = Popen(
  ['elasticsearch-7.0.0/bin/elasticsearch'], 
  stdout = PIPE, stderr = STDOUT,
  preexec_fn = lambda: os.setuid(1))

In [6]:
!curl -X GET "localhost:9200/"

{
  "name" : "7d14308a8389",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "3DEowPo8RUCG7be2-Jciqg",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [7]:
def gen_data():
  with open('/content/gdrive/My Drive/finch/es/free_chat/data/basic.txt') as f:
    for line in f:
      line = line.rstrip()
      q, a = line.split('<SEP>')
      yield {
        '_index': 'chatbot',
        'question': q,
        'answer': a,}

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
es = Elasticsearch()
print(es.ping())

es.indices.create(index='chatbot')

mapping = {
  'properties': {
    'question': {
      'type': 'text',
      'analyzer': 'ik_max_word',
      'search_analyzer': 'ik_max_word',
    }
  }
}
es.indices.put_mapping(body=mapping, index='chatbot')

helpers.bulk(es, gen_data())

True


(366, [])

In [ ]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  dsl = {
    'query': {
      'match': {
        'question': text_inp,
      }
    }
  }
  hits = es.search(index='chatbot', body=dsl)['hits']['hits']
  print('Match:', hits[0]['_source'])
  print('%.2f sec' % (time.time() - t0))
  print()

Input:爱你
Match: {'question': '爱你', 'answer': '爱你'}
0.14 sec

Input:我喜欢你
Match: {'question': '我喜欢看书', 'answer': '书是人类进步的阶梯'}
0.03 sec

Input:我爱你
Match: {'question': '我爱你', 'answer': '我也爱你'}
0.01 sec

Input:天气
Match: {'question': '天气', 'answer': '天气还行 要查具体的吗'}
0.01 sec

Input:几点了
Match: {'question': '现在几点了', 'answer': '要查一下现在的具体时间吗'}
0.01 sec

Input:几岁了
Match: {'question': '你几岁了', 'answer': '你猜呢'}
0.01 sec

Input:笑话
Match: {'question': '笑话', 'answer': '白天文明 但不精神 晚上精神 但不文明'}
0.01 sec

Input:天气好
Match: {'question': '天气', 'answer': '天气还行 要查具体的吗'}
0.01 sec

Input:天气糟糕
Match: {'question': '天气太糟糕了', 'answer': '最好不要出门'}
0.01 sec

Input:冷
Match: {'question': '冷', 'answer': '多穿衣服哈'}
0.01 sec

Input:热
Match: {'question': '热', 'answer': '对啊, 热死人了'}
0.01 sec

Input:热死了
Match: {'question': '热', 'answer': '对啊, 热死人了'}
0.01 sec

Input:激动死了
Match: {'question': '太激动了', 'answer': '发生什么好事了 说来听听'}
0.01 sec

Input:我伤心了
Match: {'question': '我好伤心', 'answer': '怎么了 说来听听'}
0.01 sec

Input:厉害了
Match: {'question': '